In [21]:
import os
import json
import requests
import textwrap
from bs4 import BeautifulSoup
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4


def scraping_do_texto():
    link = input('Digite o link: ')
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }
    requisicao = requests.get(link, headers=headers)
    
    if requisicao.status_code == 200:
        site = BeautifulSoup(requisicao.text, 'html.parser')
        paragrafos = site.find_all('p')
        texto = ' '.join([paragrafo.get_text() for paragrafo in paragrafos])
        return texto
    else:
        print(f"Erro ao acessar o site: {requisicao.status_code}")
        return None

def analise_texto(texto, chave_api):
    headers = {'Authorization':f'Bearer {API_Key}',
               'Content-Type': 'application/json'
    }
    
    url = 'https://api.openai.com/v1/chat/completions'
    id_modelo = 'gpt-4o-mini'
    
    messages = [{'role': 'user','content': 'A partir do texto, quero uma análise mais sofisticado sobre o assunto'}]
    messages.append({'role': 'user', 'content': texto})
    
    body_message = {
       'model': id_modelo,
       'messages': messages,
       'temperature': 1,
       'max_tokens': 1000
    }
    
    requisicao = requests.post(url, headers=headers, json=body_message)
    response_data = requisicao.json()

    # Extraindo o conteúdo da resposta
    try:
        resposta = response_data['choices'][0]['message']['content'].strip()
    except (KeyError, IndexError):
        print('Erro ao extrair resposta da API.')
        resposta = 'Nenhuma resposta obtida.'
    return resposta


    # Caminho para salvar o PDF
def salvar_pdf(texto, resposta, pasta_destino):
    os.makedirs(pasta_destino, exist_ok=True)  # Cria a pasta se não existir
    nome_arquivo = os.path.join(pasta_destino, 'resposta_openai.pdf')
    
    # Configuração de margens e layout
    margem_esquerda = 2 * cm
    margem_direita = 2 * cm
    margem_superior = 2 * cm
    margem_inferior = 2 * cm
    largura, altura = A4
        
    # Criando o PDF
    c = canvas.Canvas(nome_arquivo, pagesize=A4)
    
    # Título
    c.setFont('Helvetica-Bold', 20)
    c.drawCentredString(largura / 2, altura - 100, 'Resposta da OpenAI')
    
    # Subtítulo
    c.setFont('Helvetica-Bold', 14)
    c.drawCentredString(largura / 2, altura - 130, f"Pergunta: {texto}")
    
    # Corpo do texto
    c.setFont('Helvetica', 12)
    text_object = c.beginText(margem_esquerda, altura - 180 - 70)
    text_object.setLeading(14)  # Espaçamento entre linhas
    
    # Quebrando o texto para respeitar a largura definida
    wrapped_text = textwrap.wrap(resposta, width=95)
    
    for linha in wrapped_text:
        # Verificando se o texto ultrapassa o limite inferior
        if text_object.getY() <= margem_inferior:
            c.drawText(text_object)
            c.showPage()
            text_object = c.beginText(margem_esquerda, altura - margem_superior)
            text_object.setFont('Helvetica', 12)
            text_object.setLeading(16)
        text_object.textLine(linha)
    
    c.drawText(text_object)
    c.save()
    
    print(f"PDF salvo em: {nome_arquivo}")

if __name__ == '__main__':
    chave_api = {'Authorization':f'Bearer {API_Key}',
               'Content-Type': 'application/json'
    }  # Substitua pela sua chave de API
    pasta_destino = 'Z:/Controladoria e Planejamento Financeiro/03 - Projetos\Projeto Release'  # Substitua pelo caminho desejado
    
    texto = scraping_do_texto()
    if texto:
        resposta = analise_texto(texto, chave_api)
        salvar_pdf(texto, resposta, pasta_destino)

Digite o link:  https://qedu.org.br/municipio/3106200-belo-horizonte


PDF salvo em: Z:/Controladoria e Planejamento Financeiro/03 - Projetos\Projeto Release\resposta_openai.pdf
